In [17]:
# Input data processing
import pandas as pd
import numpy as np
df = pd.read_table('iris.data', header=None)
raw_data = np.array(df)
pro_data = [[],[],[]]
for item in raw_data:
    pro_item = str(item[0]).split(',')
    pro_item = [eval(pro_item[0]),eval(pro_item[1]),eval(pro_item[2]),eval(pro_item[3]),pro_item[4]]
    if pro_item[4] == 'Iris-setosa':    
        pro_item[4] = 0
        pro_data[0].append(pro_item)
    if pro_item[4] == 'Iris-versicolor': 
        pro_item[4] = 1
        pro_data[1].append(pro_item)
    if pro_item[4] == 'Iris-virginica':  
        pro_item[4] = 2
        pro_data[2].append(pro_item)
pro_data = np.array(pro_data)
train_ratio = 0.8
def train_test(all_data, train_ratio):
    train_len = round(len(all_data)*train_ratio)
    np.random.shuffle(all_data)
    train_data = all_data[0:train_len]
    test_data = all_data[train_len:len(all_data)]
    return[train_data,test_data]
# print(train_test(pro_data[0], train_ratio))
m = 4 # attribute
n = 3 # types
# save training and testing data
training_set = [[[] for col in range(m)] for row in range(n)]
testing_set = [[] for row in range(n)]
# for each type
for ii in range(n):
    x = train_test(pro_data[ii],train_ratio)
    testing_set[ii] = x[1][:,0:4]
#     for each attribute
    for jj in range(m):
        training_set[ii][jj] = x[0][:,jj]

In [18]:
testing_set

[array([[5.4, 3.4, 1.7, 0.2],
        [5. , 3.5, 1.3, 0.3],
        [5.2, 3.5, 1.5, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [5.2, 3.4, 1.4, 0.2],
        [4.6, 3.2, 1.4, 0.2],
        [5.1, 3.5, 1.4, 0.2],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3.6, 1.4, 0.2]]),
 array([[6.6, 3. , 4.4, 1.4],
        [6.2, 2.2, 4.5, 1.5],
        [6.6, 2.9, 4.6, 1.3],
        [5.5, 2.4, 3.8, 1.1],
        [6. , 2.9, 4.5, 1.5],
        [5.9, 3.2, 4.8, 1.8],
        [5.7, 2.6, 3.5, 1. ],
        [6.3, 2.3, 4.4, 1.3],
        [6. , 3.4, 4.5, 1.6],
        [5.4, 3. , 4.5, 1.5]]),
 array([[6.5, 3.2, 5.1, 2. ],
        [6.9, 3.1, 5.1, 2.3],
        [6.5, 3. , 5.2, 2. ],
        [6.3, 3.3, 6. , 2.5],
        [6. , 2.2, 5. , 1.5],
        [6.3, 2.5, 5. , 1.9],
        [6.1, 2.6, 5.6, 1.4],
        [7.2, 3. , 5.8, 1.6],
        [4.9, 2.5, 4.5, 1.7],
        [6.3, 3.4, 5.6, 2.4]])]

In [4]:
# EM iteration to estimate GMM coefficients with training data set
from sklearn.mixture import GaussianMixture
# GMM coefficients
components_num = 3
weights = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
means = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
covariances = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
for ii in range(n):
    for jj in range(m):
        gmm_coefficients = GaussianMixture(n_components=3).fit(training_set[ii][jj].reshape(-1, 1))
        for item in range(components_num):
            weights[ii][jj][item] = gmm_coefficients.weights_[item]
            means[ii][jj][item] = gmm_coefficients.means_[item][0]
            covariances[ii][jj][item] = gmm_coefficients.covariances_[item][0]

c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known

In [3]:
# Plot GMM curves


1

In [81]:
# Generate BPAs
def calc_Gaussian(mean,cov,x):
    return (1/np.sqrt(2*np.pi*cov))*np.exp(-np.square(x-mean)/(2*cov))

def calc_mix_Gaussian(weights,means,covs,x):
    components_num = len(weights)
    prob = 0
    for item in range(components_num):
        prob += weights[item]*calc_Gaussian(means[item],covs[item],x)
    return prob[0]

def generate_BPAs(x, weights, means, covariances):
    # x: input testing data (4 atrributes)
    attributes_num = len(x)
    types_num = len(means)
    # BPAs for each attribute
    BPAs = [[] for attribute in range(attributes_num)]
    for attribute in range(attributes_num):
        attribute_value = x[attribute]
        # calculate GMM distribution value
        f = [[] for type in range(types_num)]
        for type in range(types_num):
            f[type] = calc_mix_Gaussian(weights[type][attribute],means[type][attribute],covariances[type][attribute],attribute_value)
        f = np.array(f)
        # calculate pi_0 and pi_1
        pi1 = f/max(f)
        pi0 = np.ones(types_num)-pi1
        # calculate BPA and record
        dimension = 2**types_num
        BPA = [[] for ii in range(dimension)]
        for ii in range(dimension):
            # binary represenation
            bits = "{:0>10b}".format(ii)
            # mass
            m = 1
            for jj in range(types_num):
                bit = bits[-(jj+1)]
                if bit == '1':
                    m *= pi1[jj]
                if bit == '0':
                    m *= pi0[jj]
            BPA[ii] = m
        BPAs[attribute] = BPA
    return BPAs

x = generate_BPAs(testing_set[0][0], weights, means, covariances)
        

[0.0,
 1.0,
 0.0,
 4.1903814209289186e-42,
 0.0,
 2.313859356216941e-22,
 0.0,
 9.695953256934017e-64]

In [6]:
# Combining BPAs on quantum circuits and conduct measurement


In [7]:
# Decision-making by the combined BPA


In [ ]:
# Testing